In [2]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import stats
from sklearn.cluster import KMeans 
from sklearn import metrics 
from scipy.spatial.distance import cdist 
#from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn import tree
import graphviz
import os
os.environ["PATH"] += os.pathsep + 'F:/Data Analytics-MS!/Integrated Experiential Learn/Week 5 & 6/release/bin'
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import recall_score
#from imblearn.over_sampling import SMOTE

In [4]:
df1 = pd.read_csv('F:/Data Analytics-MS!/Integrated Experiential Learn/Week 5 & 6/indicator_pairs_data updated.csv')
df2 = pd.read_csv('F:/Data Analytics-MS!/Integrated Experiential Learn/Week 5 & 6/obfuscated_demo_data2.csv')

In [5]:
df = pd.merge(df1, df2, on='employee_id', how = 'inner')

In [6]:
df = df[df['alert_type'] != 'Atomic'] 

In [7]:
df['Dates'] = pd.to_datetime(df['ge_hire_date']).dt.date
df['Time'] = pd.to_datetime(df['ge_hire_date']).dt.time
df['year'] = pd.DatetimeIndex(df['Dates']).year
df['Tenure'] = df.apply(lambda row: 2020 - row.year, axis = 1) 
df['year'] = df['year'].fillna(0).astype(int)
df['Tenure'] = df['Tenure'].fillna(0).astype(int)

In [8]:
df = df.drop(['alert_id_fk', 'insert_date', 'employee_id', 'alert_escalation_date', 'ge_hire_date', 'city', 'state_name', 'country_name', 'alert_category', 'year', 'Time', 'Dates'], axis=1)

In [9]:
#Splitting indicator column into multiple columns
df[['indicator_pairs', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21']] = df.indicator_pairs.str.split("/",expand=True)

In [17]:
#export_csv = df.to_csv (r'F:\Data Analytics-MS!\Integrated Experiential Learn\Final Submissions\heat_data.csv', index = None, header=True)

In [10]:
dfd = df.copy()
dfw = df.copy()
dfm = df.copy()
dfd = dfd[dfd['alert_type'] != 'Atomic'] 
dfd = dfd[dfd['alert_type'] == 'Daily_Heat'] 
dfw = dfw[dfw['alert_type'] != 'Atomic'] 
dfw = dfw[dfw['alert_type'] == 'Weekly_Heat'] 
dfm = dfm[dfm['alert_type'] != 'Atomic'] 
dfm = dfm[dfm['alert_type'] == 'Monthly_Heat'] 

In [11]:
#Target Variable
dfd['classification'] = np.where((dfd['classification'] == 'TP/HIGH'), 1, 0)
#split dataset in features and target variable
feature_cols = ['score', 'owner_name', 'hru', 'person_type', 'person_status', 'alert_type', 'risk_factor', 'Tenure', 'indicator_pairs']
X = dfd[feature_cols] # Features
y = dfd.classification #Target Variable

#One-hot Encoding
X = pd.get_dummies(X)

#Splitting into training and testing sets
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [12]:
#Training the Algorithm

classifier = RandomForestClassifier(n_estimators=20, random_state=0)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

In [13]:
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[3124   46]
 [  68   38]]
             precision    recall  f1-score   support

          0       0.98      0.99      0.98      3170
          1       0.45      0.36      0.40       106

avg / total       0.96      0.97      0.96      3276

0.965201465201


In [14]:
threshold = 0.5

predicted_proba = classifier.predict_proba(X_test)
predicted = (predicted_proba [:,1] >= threshold).astype('int')

accuracy = accuracy_score(y_test, predicted)
print(accuracy)

print(confusion_matrix(y_test,predicted))

0.965506715507
[[3124   46]
 [  67   39]]


In [15]:
threshold = 0.25

predicted_proba = classifier.predict_proba(X_test)
predicted = (predicted_proba [:,1] >= threshold).astype('int')

accuracy = accuracy_score(y_test, predicted)
print(accuracy)

print(confusion_matrix(y_test,predicted))

0.961843711844
[[3094   76]
 [  49   57]]


In [16]:
threshold = 0.167

predicted_proba = classifier.predict_proba(X_test)
predicted = (predicted_proba [:,1] >= threshold).astype('int')

accuracy = accuracy_score(y_test, predicted)
print(accuracy)

print(confusion_matrix(y_test,predicted))

0.959096459096
[[3076   94]
 [  40   66]]


In [17]:
threshold = 0.023

predicted_proba = classifier.predict_proba(X_test)
predicted = (predicted_proba [:,1] >= threshold).astype('int')

accuracy = accuracy_score(y_test, predicted)
print(accuracy)

print(confusion_matrix(y_test,predicted))

0.900183150183
[[2862  308]
 [  19   87]]


In [18]:
threshold = 0.001

predicted_proba = classifier.predict_proba(X_test)
predicted = (predicted_proba [:,1] >= threshold).astype('int')

accuracy = accuracy_score(y_test, predicted)
print(accuracy)

print(confusion_matrix(y_test,predicted))

0.892246642247
[[2835  335]
 [  18   88]]


In [22]:
#Oversampling for daily
sm = SMOTE(random_state=12)
X_train_res, y_train_res = sm.fit_sample(X_train, y_train)
#Training the Algorithm
classifier3 = RandomForestClassifier(n_estimators=30, random_state=12)
classifier3.fit(X_train_res, y_train_res)
print ('Validation Results')
print (classifier3.score(X_test, y_test))
print (recall_score(y_test, classifier3.predict(X_test)))
print ('\n Test Results')
print (classifier3.score(X, y))
print (recall_score(y, classifier3.predict(X)))

Validation Results
0.9615384615384616
0.4528301886792453

 Test Results
0.9774711520849869
0.7473684210526316


In [19]:
#Target Variable
dfm['classification'] = np.where((dfm['classification'] == 'TP/HIGH'), 1, 0)
#split dataset in features and target variable
feature_cols = ['score', 'owner_name', 'hru', 'person_type', 'person_status', 'alert_type', 'risk_factor', 'Tenure', 'indicator_pairs']
X = dfm[feature_cols] # Features
y = dfm.classification #Target Variable

#One-hot Encoding
X = pd.get_dummies(X)

#Splitting into training and testing sets
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

#Training the Algorithm

classifier = RandomForestClassifier(n_estimators=20, random_state=0)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[2151   41]
 [  52   42]]
             precision    recall  f1-score   support

          0       0.98      0.98      0.98      2192
          1       0.51      0.45      0.47        94

avg / total       0.96      0.96      0.96      2286

0.959317585302


In [20]:
threshold = 0.5

predicted_proba = classifier.predict_proba(X_test)
predicted = (predicted_proba [:,1] >= threshold).astype('int')

accuracy = accuracy_score(y_test, predicted)
print(accuracy)

print(confusion_matrix(y_test,predicted))

0.958880139983
[[2150   42]
 [  52   42]]


In [21]:
threshold = 0.25

predicted_proba = classifier.predict_proba(X_test)
predicted = (predicted_proba [:,1] >= threshold).astype('int')

accuracy = accuracy_score(y_test, predicted)
print(accuracy)

print(confusion_matrix(y_test,predicted))

0.959317585302
[[2139   53]
 [  40   54]]


In [22]:
threshold = 0.167

predicted_proba = classifier.predict_proba(X_test)
predicted = (predicted_proba [:,1] >= threshold).astype('int')

accuracy = accuracy_score(y_test, predicted)
print(accuracy)

print(confusion_matrix(y_test,predicted))

0.959755030621
[[2132   60]
 [  32   62]]


In [23]:
threshold = 0.023

predicted_proba = classifier.predict_proba(X_test)
predicted = (predicted_proba [:,1] >= threshold).astype('int')

accuracy = accuracy_score(y_test, predicted)
print(accuracy)

print(confusion_matrix(y_test,predicted))

0.874453193351
[[1914  278]
 [   9   85]]


In [24]:
threshold = 0.001

predicted_proba = classifier.predict_proba(X_test)
predicted = (predicted_proba [:,1] >= threshold).astype('int')

accuracy = accuracy_score(y_test, predicted)
print(accuracy)

print(confusion_matrix(y_test,predicted))

0.866579177603
[[1896  296]
 [   9   85]]


In [24]:
#Oversampling for monthly
sm = SMOTE(random_state=12)
X_train_res, y_train_res = sm.fit_sample(X_train, y_train)
#Training the Algorithm
classifier1 = RandomForestClassifier(n_estimators=30, random_state=12)
classifier1.fit(X_train_res, y_train_res)
print ('Validation Results')
print (classifier1.score(X_test, y_test))
print (recall_score(y_test, classifier1.predict(X_test)))
print ('\n Test Results')
print (classifier1.score(X, y))
print (recall_score(y, classifier1.predict(X)))

Validation Results
0.9558180227471567
0.425531914893617

 Test Results
0.9749737486874344
0.75


In [25]:
#Target Variable
dfw['classification'] = np.where((dfw['classification'] == 'TP/HIGH'), 1, 0)
#split dataset in features and target variable
feature_cols = ['score', 'owner_name', 'hru', 'person_type', 'person_status', 'alert_type', 'risk_factor', 'Tenure', 'indicator_pairs']
X = dfw[feature_cols] # Features
y = dfw.classification #Target Variable

#One-hot Encoding
X = pd.get_dummies(X)

#Splitting into training and testing sets
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

#Training the Algorithm

classifier = RandomForestClassifier(n_estimators=20, random_state=0)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[2181   36]
 [  56   64]]
             precision    recall  f1-score   support

          0       0.97      0.98      0.98      2217
          1       0.64      0.53      0.58       120

avg / total       0.96      0.96      0.96      2337

0.960633290543


In [26]:
threshold = 0.5

predicted_proba = classifier.predict_proba(X_test)
predicted = (predicted_proba [:,1] >= threshold).astype('int')

accuracy = accuracy_score(y_test, predicted)
print(accuracy)

print(confusion_matrix(y_test,predicted))

0.960205391528
[[2180   37]
 [  56   64]]


In [27]:
threshold = 0.25

predicted_proba = classifier.predict_proba(X_test)
predicted = (predicted_proba [:,1] >= threshold).astype('int')

accuracy = accuracy_score(y_test, predicted)
print(accuracy)

print(confusion_matrix(y_test,predicted))

0.963200684638
[[2170   47]
 [  39   81]]


In [28]:
threshold = 0.167

predicted_proba = classifier.predict_proba(X_test)
predicted = (predicted_proba [:,1] >= threshold).astype('int')

accuracy = accuracy_score(y_test, predicted)
print(accuracy)

print(confusion_matrix(y_test,predicted))

0.966623876765
[[2166   51]
 [  27   93]]


In [29]:
threshold = 0.023

predicted_proba = classifier.predict_proba(X_test)
predicted = (predicted_proba [:,1] >= threshold).astype('int')

accuracy = accuracy_score(y_test, predicted)
print(accuracy)

print(confusion_matrix(y_test,predicted))

0.91270860077
[[2020  197]
 [   7  113]]


In [30]:
threshold = 0.001

predicted_proba = classifier.predict_proba(X_test)
predicted = (predicted_proba [:,1] >= threshold).astype('int')

accuracy = accuracy_score(y_test, predicted)
print(accuracy)

print(confusion_matrix(y_test,predicted))

0.908857509628
[[2011  206]
 [   7  113]]


In [26]:
#Oversampling for weekly
sm = SMOTE(random_state=12)
X_train_res, y_train_res = sm.fit_sample(X_train, y_train)
#Training the Algorithm
classifier2 = RandomForestClassifier(n_estimators=30, random_state=12)
classifier2.fit(X_train_res, y_train_res)
print ('Validation Results')
print (classifier2.score(X_test, y_test))
print (recall_score(y_test, classifier2.predict(X_test)))
print ('\n Test Results')
print (classifier2.score(X, y))
print (recall_score(y, classifier2.predict(X)))

Validation Results
0.9619169875909286
0.5833333333333334

 Test Results
0.9797962503210341
0.8293135435992579
